In [1]:
# !pip install wandb

# import wandb
# import os
# wandb.login(key="2ba3e7324f00984d9cc7150478390d2c569ba8cf")

# wandb.init(project="Methembe-GNN5", name='ST-GAT_model')

In [2]:
from google.colab import drive
drive.mount('/content/drive')


%cd /content/drive/MyDrive/ST-GAT


Mounted at /content/drive
/content/drive/MyDrive/ST-GAT


In [3]:
!pip install torch_geometric

import random
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch_geometric
from torch_geometric.nn.conv import GATConv
import torch.nn.functional as F
from load_data import *
from utils import *
from stgat import *

torch.manual_seed(2333)
torch.cuda.manual_seed(2333)
np.random.seed(2333)
random.seed(2333)
torch.backends.cudnn.deterministic = True


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


matrix_path = "./dataset/03062021_M_96.csv"
data_path = "./dataset/03062021_T_adj_96.csv"
save_path = "./save/GAT_model_2.pt"

day_slot = 288
n_train, n_val, n_test = 69, 15, 15

n_his = 24
n_pred = 12
n_route = 96
Ks, Kt = 3, 3
blocks = [[1, 32, 64], [64, 32, 128]]
drop_prob = 0.31

batch_size = 147
epochs = 100
lr = 0.0503
weight_decay = 0.03739

train, val, test = load_data(data_path, n_train * day_slot, n_val * day_slot)
scaler = StandardScaler()
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

x_train, y_train = data_transform(train, n_his, n_pred, day_slot, device)
x_val, y_val = data_transform(val, n_his, n_pred, day_slot, device)
x_test, y_test = data_transform(test, n_his, n_pred, day_slot, device)

train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00


In [4]:
def calculate_edge_index_from_adj_matrix(csv_file):
  """
  Calculates the edge index from an adjacency matrix in a CSV file.

  Args:
      csv_file (str): Path to the CSV file containing the adjacency matrix.

  Returns:
      numpy.ndarray: A 2D NumPy array of shape (num_edges, 2) representing the edge index.
                        - Each row contains [source_node_index, target_node_index] for an edge.
  """
  # Read the adjacency matrix
  adj_matrix = pd.read_csv(csv_file, header=None).values

  # Find non-zero elements (assuming they represent edges)
  edges = adj_matrix.nonzero()

  # Combine source and target node indices into a single 2D array
  edge_index = np.vstack([edges[0], edges[1]])

  edge_index_tensor = torch.LongTensor(edge_index)

  return edge_index_tensor

# Example usage
# csv_file = "./dataset/03062021_M_96.csv"
edge_index = calculate_edge_index_from_adj_matrix(matrix_path)

edge_index = edge_index.to(device)

print("Edge Index:", edge_index.shape)


Edge Index: torch.Size([2, 8624])


In [5]:
# Example usage
in_features = 96  # Assuming your nodes have 10 features
hidden_channels = 96
num_layers = 2
out_features = 96  # Number of output features for each node after prediction



model = BasicGAT(in_features, hidden_channels, num_layers, out_features)
model = model.to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)


In [6]:
for epoch in range(1, epochs + 1):
    model.train()
    final_loss ,train_loss , n , test_n = 0.0 , 0.0 , 0 , 0
    mape_batch , mae_batch , rmse_batch = 0.0 , 0.0 , 0.0

    mape_test_batch , mae_test_batch , rmse_test_batch = 0.0 , 0.0 , 0.0
    test_loss = 0.0


    for x, y in train_iter:
        # print(x.size())
        y_pred = model(x , edge_index)
        loss_value = loss(y_pred, y)
        loss_value.backward()
        optimizer.zero_grad()
        optimizer.step()
        train_loss += loss_value.item()
        n += y.shape[0]

        x_std = torch.std(x)
        x_mean = torch.mean(x)
        y = z_inverse(y , x_mean , x_std)
        y_pred = z_inverse(y_pred , x_mean , x_std)

        mape_batch += MAPE(y, y_pred)
        mae_batch += MAE(y, y_pred)
        rmse_batch += RMSE(y, y_pred)

    for x, y in test_iter:
        y_pred = model(x , edge_index)
        test_loss = loss(y_pred, y)
        test_loss.backward()
        optimizer.zero_grad()
        optimizer.step()

        y_pred = z_inverse(y_pred , x_mean , x_std)
        y = z_inverse(y , x_mean , x_std)

        test_n += y.shape[0]

        mape_test_batch += MAPE(y, y_pred)
        mae_test_batch += MAE(y, y_pred)
        rmse_test_batch += RMSE(y, y_pred)

    scheduler.step()

    # Calculate MAPE, MAE, RMSE after testing each epoch
    mape_test = mape_test_batch/test_n
    mae_test = mae_test_batch/test_n
    rmse_test = rmse_test_batch/test_n
    test_loss = test_loss/test_n

    # Calculate MAPE, MAE, RMSE after training each epoch
    mape_ = mape_batch/n
    mae_ = mae_batch/n
    rmse_ = rmse_batch/n
    final_loss = train_loss/n

    # wandb.log({"Training_Loss": loss_value.item() , "Testing_Loss": test_loss.item()  ,"Test_RMSE": rmse_test ,"Test_MAE": mae_test , "Test_MAPE":mape_test ,
    #          "RMSE": rmse_, "MAE": mae_ , "MAPE": mape_*100 }, step=epoch)

    if epoch % 5 == 0:
        torch.save(model.state_dict(), save_path)

    print(f"Epoch number: {epoch} - Training Loss: {final_loss} - MAE: {mae_} - MAPE: {mape_*100} - RMSE: {rmse_} ")


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch number: 1 - Training Loss: 0.010929209209358446 - MAE: 0.006650480907410383 - MAPE: -0.3816549777984619 - RMSE: 0.009119877591729164 
Epoch number: 2 - Training Loss: 0.010928010936454515 - MAE: 0.006653799209743738 - MAPE: -0.34926339983940125 - RMSE: 0.009121833369135857 
Epoch number: 3 - Training Loss: 0.010930932735177177 - MAE: 0.006655038800090551 - MAPE: -0.28191566467285156 - RMSE: 0.009124530479311943 
Epoch number: 4 - Training Loss: 0.010928663982383272 - MAE: 0.006653320975601673 - MAPE: 0.20068466663360596 - RMSE: 0.009120623581111431 
Epoch number: 5 - Training Loss: 0.010925656906373109 - MAE: 0.0066506871953606606 - MAPE: -1.777777075767517 - RMSE: 0.009118027985095978 
Epoch number: 6 - Training Loss: 0.010927309645487625 - MAE: 0.006652703508734703 - MAPE: -0.25022804737091064 - RMSE: 0.009121185168623924 
Epoch number: 7 - Training Loss: 0.01092936779998344 - MAE: 0.0066557084210217 - MAPE: -0.3656529486179352 - RMSE: 0.009122596122324467 
Epoch number: 8 - Tr

In [7]:
best_model = BasicGAT(in_features, hidden_channels, num_layers, out_features).to(device)
best_model.load_state_dict(torch.load(save_path))

<All keys matched successfully>